<a href="https://colab.research.google.com/github/ochoa-josue/techwise-deepfake-classification/blob/feature%2Fgan-model/gan_discriminator_deepfake_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow matplotlib tensorflow-datasets ipywidgets

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory #load image datasets
from matplotlib import pyplot as plt
from tensorflow.keras.models import Sequential, load_model #sequential api for generator and discriminator
from tensorflow.keras.layers import Conv2D, Dense, Flatten, Reshape, LeakyReLU, Dropout, UpSampling2D, BatchNormalization
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.regularizers import l2
from tensorflow.keras.preprocessing import image
import numpy as np

### **Retrieve and Preprocess Dataset**

In [ ]:
from google.colab import drive
import os

drive.mount('/content/drive')
os.chdir("/content/drive/My Drive/conforming-images-subset") #change current working directory to specified directory

Mounted at /content/drive


In [ ]:
train_dir = "/content/drive/My Drive/conforming-images-subset/train"
valid_dir = "/content/drive/My Drive/conforming-images-subset/valid"
test_dir = "/content/drive/My Drive/conforming-images-subset/test"

In [ ]:
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal'),
    tf.keras.layers.experimental.preprocessing.RandomRotation(0.05),
    tf.keras.layers.experimental.preprocessing.RandomZoom(0.05),
    tf.keras.layers.experimental.preprocessing.RandomTranslation(height_factor=0.05, width_factor=0.05),
])

def preprocess(images, labels):
  images = tf.cast(images, tf.float32) #convert tensor img to tf.float32 type
  images = (images / 127.5) - 1 #normalize data between range -1 to 1
  return images, labels

def preprocess_with_augmentation(images, labels):
  images = data_augmentation(images, training=True)
  images = tf.cast(images, tf.float32) #convert tensor img to tf.float32 type
  images = (images / 127.5) - 1 #normalize data between range -1 to 1
  return images, labels


In [ ]:
train_dataset = image_dataset_from_directory(train_dir, label_mode='binary', image_size=(256,256), batch_size=32) #image anomaly detection
valid_dataset = image_dataset_from_directory(valid_dir, label_mode='binary', image_size=(256,256), batch_size=32)
test_dataset = image_dataset_from_directory(test_dir, label_mode='binary', image_size=(256,256), batch_size=32)

train_dataset = train_dataset.map(preprocess_with_augmentation)
valid_dataset = valid_dataset.map(preprocess)
test_dataset = test_dataset.map(preprocess)

Found 10030 files belonging to 2 classes.
Found 50 files belonging to 2 classes.
Found 4000 files belonging to 2 classes.


### **Build GAN**

In [ ]:
class GAN(tf.keras.Model):
  def __init__(self, generator, discriminator):
    super(GAN, self).__init__() #invoking constructor of tf.keras.Model to initialize GAN proxy object
    self.generator = generator
    self.discriminator = discriminator

  def compile(self, g_optimizer, d_optimizer, loss_function):
    super(GAN, self).compile() #ensure compilation of parent class is executed
    self.g_optimizer = g_optimizer
    self.d_optimizer = d_optimizer
    self.loss_function = loss_function

  def train_step(self, real_images): #override parent train_step when calling to TensorFlow fit function
    batch_size = tf.shape(real_images)[0] #retrieving batch size
    noise_dim = 128 #128 dimensions

    #generate fake images
    random_noise = tf.random.normal(shape=(batch_size, noise_dim)) #create random noise vectors with Gaussian distribution
    fake_images = self.generator(random_noise, training=True)

    #train discriminator
    with tf.GradientTape() as disc_tape: #records operations on tensors; thereafter performs automatic differentiation, in other words, compute derivatives or gradients of functions
        real_output = self.discriminator(real_images, training=True)
        fake_output = self.discriminator(fake_images, training=True)

        #calculate loss
        real_loss = self.loss_function(tf.ones_like(real_output), real_output)
        fake_loss = self.loss_function(tf.zeros_like(fake_output), fake_output)

        #total discriminator loss
        disc_loss = real_loss + fake_loss

    #calculate gradients for discriminator
    gradients_of_discriminator = disc_tape.gradient(disc_loss, self.discriminator.trainable_variables)

    #apply gradients to the optimizer
    self.d_optimizer.apply_gradients(zip(gradients_of_discriminator, self.discriminator.trainable_variables))

    #train generator
    with tf.GradientTape() as gen_tape:
      fake_images = self.generator(random_noise, training=True) #generate fake images
      fake_output = self.discriminator(fake_images, training=True) #disc predict fake images
      gen_loss = self.loss_function(tf.ones_like(fake_output), fake_output) #show generator loss; maximimize loss to optimize generator

    #calculate gradients for generator
    gradients_of_generator = gen_tape.gradient(gen_loss, self.generator.trainable_variables)
    #apply gradients to the optimizer
    self.g_optimizer.apply_gradients(zip(gradients_of_generator, self.generator.trainable_variables))

    return {'disc_loss': disc_loss, 'gen_loss': gen_loss}


In [ ]:
def build_generator():
  model = Sequential()
  model.add(Dense(7*7*128, input_dim=128)) #intakes 128-dimesional input of random noise to output tensor shape
  model.add(LeakyReLU(0.2)) #introduce non-linearlity for continued learning
  model.add(BatchNormalization(momentum=0.8)) #stabilize training process by tracking moving averages overtime; faster convergence
  model.add(Reshape((7, 7, 128))) #reshapes into 4D tensor which inclues batch size, height, width, and channels

  #upsampling block 1
  model.add(UpSampling2D()) #double dimensions of the input tensor for higher resolutions from low-dimensional noise vectors
  model.add(Conv2D(128, 5, padding='same')) #adds a convolutional layer wit 256 filters, 5x5 kernel size, and same padding to preserve spatial dimensions as input image
  model.add(LeakyReLU(0.2))
  model.add(BatchNormalization(momentum=0.8))

  #upsampling block 2
  model.add(UpSampling2D())
  model.add(Conv2D(64, 5, padding='same')) #reduce filters to prevent overfitting and allow feature abstraction e.g. capturing simple patterns (edges, textures)
  model.add(LeakyReLU(0.2))
  model.add(BatchNormalization(momentum=0.8))

  #simplify model
  # #convolution block 1
  # model.add(Conv2D(128, 4, padding='same'))
  # model.add(LeakyReLU(0.2))

  #output layer adjustment; output layer is tanh activation
  model.add(Conv2D(3, 4, padding='same', activation='tanh')) #tanh activation for data normalized between -1 and 1 for RGB images

  return model

In [ ]:
def build_discriminator(input_shape=(256, 256, 3)):
  model = Sequential([
      #convolution block 1
      Conv2D(32, kernel_size=5, strides=2, input_shape=input_shape, padding="same", kernel_regularizer=l2(0.01)), #stride reduces spatial dimensions by half
      LeakyReLU(alpha=0.2),
      Dropout(0.3), #prevent overfitting where 30% of input units or features are randomly set to zero

      #convolution block 2
      Conv2D(64, kernel_size=5, strides=2, padding="same", kernel_regularizer=l2(0.01)),
      LeakyReLU(alpha=0.2),
      Dropout(0.3),

      #simplify model
      # #convolution block 3
      # Conv2D(256, kernel_size=5, strides=2, padding="same"),
      # LeakyReLU(alpha=0.2),
      # Dropout(0.3),

      #flatten and output layer
      Flatten(), #flatten into a 1D tensor for easier integration with fully connected layers
      Dense(1, kernel_regularizer=l2(0.01)) #indicate whether image is real or fake
  ])

  return model

### **Data Visualization and Compiling GAN Model**

In [ ]:
# generator = build_generator()
# g_optimizer = tf.keras.optimizers.Adam(1e-4) #find best convergence along with optimal learning rate
# noise_dim = 128
# gan_model = GAN(generator, discriminator, noise_dim) #create GAN
# gan_model.compile(g_optimizer=g_optimizer, d_optimizer=d_optimizer) #compile GAN
# gan_model.fit(train_dataset, epochs=50, validation_data=valid_dataset, callbacks=[checkpoint_cb, early_stopping_cb])

discriminator = build_discriminator()
d_optimizer = tf.keras.optimizers.Adam(1e-4)
loss_function = tf.keras.losses.BinaryCrossentropy(from_logits=True)
discriminator.compile(optimizer=d_optimizer, loss=loss_function, metrics=['accuracy', AUC(name='auc')])

checkpoint_cb = ModelCheckpoint("best_discriminator", save_best_only=True, monitor='val_loss') #callback to save the best model checkpoint during training
early_stopping_cb = EarlyStopping(patience=10, restore_best_weights=True, monitor='val_loss') #stop training if no improvement at 10 consecutive epochs

#train the discriminator on the dataset
history = discriminator.fit(train_dataset, epochs=50, validation_data=valid_dataset, callbacks=[checkpoint_cb, early_stopping_cb])

discriminator.save('final_discriminator_model')

Epoch 1/50
314/314 [==============================] - ETA: 0s - loss: 0.0049 - accuracy: 1.0000

314/314 [==============================] - 3340s 10s/step - loss: 0.0049 - accuracy: 1.0000 - val_loss: 1.1988e-12 - val_accuracy: 1.0000
Epoch 2/50
314/314 [==============================] - 2976s 9s/step - loss: 5.5207e-12 - accuracy: 1.0000 - val_loss: 1.1988e-12 - val_accuracy: 1.0000
Epoch 3/50
314/314 [==============================] - 2956s 9s/step - loss: 5.3902e-12 - accuracy: 1.0000 - val_loss: 1.1988e-12 - val_accuracy: 1.0000
Epoch 4/50
314/314 [==============================] - ETA: 0s - loss: 5.2236e-12 - accuracy: 1.0000

314/314 [==============================] - 2948s 9s/step - loss: 5.2236e-12 - accuracy: 1.0000 - val_loss: 1.1988e-12 - val_accuracy: 1.0000
Epoch 5/50
314/314 [==============================] - 2914s 9s/step - loss: 5.7307e-12 - accuracy: 1.0000 - val_loss: 1.1988e-12 - val_accuracy: 1.0000
Epoch 6/50
314/314 [==============================] - ETA: 0s - loss: 5.3264e-12 - accuracy: 1.0000

314/314 [==============================] - 2924s 9s/step - loss: 5.3264e-12 - accuracy: 1.0000 - val_loss: 1.1988e-12 - val_accuracy: 1.0000
Epoch 7/50
314/314 [==============================] - ETA: 0s - loss: 5.2851e-12 - accuracy: 1.0000

314/314 [==============================] - 2900s 9s/step - loss: 5.2851e-12 - accuracy: 1.0000 - val_loss: 1.1988e-12 - val_accuracy: 1.0000
Epoch 8/50
314/314 [==============================] - 2893s 9s/step - loss: 4.8021e-12 - accuracy: 1.0000 - val_loss: 1.1988e-12 - val_accuracy: 1.0000
Epoch 9/50
314/314 [==============================] - ETA: 0s - loss: 5.3457e-12 - accuracy: 1.0000

...

314/314 [==============================] - 2872s 9s/step - loss: 5.9538e-12 - accuracy: 1.0000 - val_loss: 1.1988e-12 - val_accuracy: 1.0000
Epoch 14/50
314/314 [==============================] - ETA: 0s - loss: 5.6735e-12 - accuracy: 1.0000

314/314 [==============================] - 2873s 9s/step - loss: 5.6735e-12 - accuracy: 1.0000 - val_loss: 1.1988e-12 - val_accuracy: 1.0000
Epoch 15/50
314/314 [==============================] - ETA: 0s - loss: 5.7790e-12 - accuracy: 1.0000

314/314 [==============================] - 2873s 9s/step - loss: 5.7790e-12 - accuracy: 1.0000 - val_loss: 1.1987e-12 - val_accuracy: 1.0000
Epoch 16/50
314/314 [==============================] - ETA: 0s - loss: 5.0838e-12 - accuracy: 1.0000

314/314 [==============================] - 2880s 9s/step - loss: 5.0838e-12 - accuracy: 1.0000 - val_loss: 1.1987e-12 - val_accuracy: 1.0000
Epoch 17/50
256/314 [=======================>......] - ETA: 9:11 - loss: 2.8172e-12 - accuracy: 1.0000

KeyboardInterrupt: 

In [ ]:
from sklearn.metrics import precision_recall_fscore_support, confusion_matrix
from tensorflow.keras.metrics import AUC #area under the curve for model performance (especially imbalanced datasets)
import seaborn as sns


#visualize AUC
plt.plot(history.history['auc'], label='Training AUC')
plt.plot(history.history['val_auc'], label='Validation AUC')
plt.title('Training and Validation AUC')
plt.legend()
plt.show()

#visualize training process
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.show()

#evaluate model performance
train_loss, train_accuracy = discriminator.evaluate(train_dataset)
val_loss, val_accuracy = discriminator.evaluate(valid_dataset)
print(f"Training Accuracy: {train_accuracy*100:.2f}%")
print(f"Training Accuracy: {val_accuracy*100:.2f}%")

def get_predictions_and_labels(model, dataset):
  predictions = []
  true_labels = []

  for images, labels in dataset:
    preds = model.predict(images)
    preds = (preds > 0.5).astype(np.int) #convert probabilities to binary labels
    predictions.extend(preds.flatten())
    true_labels.extend(labels.numpy().flatten())

  return np.array(predictions), np.array(true_labels)

predictions, true_labels = get_predictions_and_labels(discriminator, valid_dataset)
precision, recall, f1, _ = precision_recall_fscore_support(true_labels, predictions, average='binary')
print(f"Precision: {precision:.2f}, Recall: {recall:.2f}, F1 Score: {f1:.2f}")

cm = confusion_matrix(true_labels, predictions)
sns.heatmap(cm, annot=True, fmt="d", cmap='Blues')
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()

def predict_deepfake(image_path, model):
  img = image.load_img(image_path, target_size=(256, 256))
  img_array = image.img_to_array(img)
  img_array = np.expand_dims(img_array, axis=0) #expand dimensions
  img_array = (img_array / 127.5) - 1 #normalize from -1 to 1
  logits = model.predict(img_array)
  probabilities = tf.sigmoid(logits) #convert logits to probabilities

  return "fake" if probabilities.numpy()[0] < 0.5 else "real"

In [ ]:
saved_discriminator = load_model('final_discriminator_model.h5')

# example usage
# result = predict_deepfake('image_path.jpg', saved_discriminator)
# print("The image is: ", result)